Background:
Our client is a startup craft beer brewer, they would like to setup a distribution network of their craft beer in one of the area in Toronto. Since the supply of craft beer is limited, they would like to find out where should they setup their selling network to maximize their profit. 

Statement of Problem:
-supply and "Best Tasting Period" are limited, and the target selling price of their beers is 60% more expensive than branded beers e.g. Heineken, Budweiser, 
-beers with special flavour like herbs, sours, salty lemon, etc. (Asian Flavour) 

Based on the above, our client would like to find out a place that there is lots of bar/pubs/restaurants (especially including Asian foods restaurants as our client think that it would be an advantage if people has exposure on Asian culture) and people are willing and affordable to spend money on my beers. 

Audience: Our client (a startup craft beer brewer)

________________________________________________________________________________________
Data:
Analytic approach: we will explore the areas around Toronto, and analyze the distribution of bar/pubs/restaurants and people's spending pattern (i.e. how frequency people go to bar/pubs/restaurants)  

Data Source: FourSquare developer API will provide all the location data we need to investigate this question.

Data required: Initially we will create a bar/pubs/restaurants density measure for each area, and also a ratio of Asian foods restaurants. Then we will analyze people's spending pattern (i.e. how frequency people go to bar/pubs/restaurants)


In [1]:
#import libraries and packages 
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import requests
import urllib.request
from urllib.request import urlopen
!pip install geopy
!pip install bs4
!pip install lxml
import lxml
from bs4 import BeautifulSoup
import pickle
import pandas as pd # library for data analsysis
import statistics
from sklearn.datasets import load_iris
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

     |████████████████████████████████| 102kB 1.0MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
     |████████████████████████████████| 102kB 999kB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 5.8MB 1.0MB/s eta 0:00:01     |███▏                            | 563kB 1.0MB/s eta 0:00:06     |█████▉                          | 1.1MB 1.0MB/s eta 0:00:05     |█████████▋                      | 1.7MB 1.0MB/s eta 0:00:04


In [2]:
#Data Preparation - To extract relevant data of city of Toronto in the province of Ontario (i.e. location, neighbour, venuetype, common visiting place, etc) from FourSquare 

# get the List of postal codes of Canada from the URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url).text
#soup = BeautifulSoup(r,"lxml").find("table",class_="wikitable sortable")
soup = BeautifulSoup(r,"html.parser").find("table",class_="wikitable sortable")

table1=[]
temp = []

for tr in soup.find_all("tr"):
    # search all <tr>
    temp.clear()
    for td in tr.find_all("td"):
        #search all <td> in each <tr>
        temp.append(td.text.replace('\n',''))
    temp = [temp]
    table1 = table1 + temp
    # put the <td> into a table

# convert to dataframe
df=pd.DataFrame(table1)
df.columns = ["Postcode", "Borough", "Neighbourhood"]
df = df.drop([0], axis=0)
df.index = pd.RangeIndex(len(df.index))

#remove "Not assigned" record in Borough
df1 = df[df.Borough != 'Not assigned']

# copy value from Borough to Neighbourhood (for those Neighbourhood = Not assigned) 
for x in df1.loc[df1['Neighbourhood'] == 'Not assigned'].index:
    df1.xs(x)['Neighbourhood'] = df1.xs(x)['Borough']
    
# Generate List of city of Toronto in the province of Ontario
df2 = df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df2 = pd.DataFrame(df2)
df2=df2.reset_index()

#add column for Latitude and Longitude
df2["Latitude"] = np.nan
df2["Longitude"] = np.nan

#download Latitude and Longitude data 
data = pd.read_csv('http://cocl.us/Geospatial_data')

# add value of Latitude and Longitude data into relevant df2 records 
for x in df2['Postcode']:
    df2.loc[df2.Postcode == x, 'Latitude'] = data.loc[data['Postal Code'] == x]['Latitude']
    df2.loc[df2.Postcode == x, 'Longitude'] = data.loc[data['Postal Code'] == x]['Longitude']


In [3]:
#Get the geograpical coordinate of toronto
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# prepare datafrome for nearby benues analysis
df3=df2.reset_index()

#ready to connect to FourSquare
CLIENT_ID = 'PYJBJMLRS1IGVZAYUO5FS0YSPJ4X2FQ0AC1LAXNWDUUPAXXD' # Foursquare ID
CLIENT_SECRET = 'JYGPSCVR3YAB425PFX141KZOZN31IH1POGC2FZRVOTYME210' # Foursquare Secret
VERSION = '201707023' # Foursquare API version

# define function to get nearby venues

def getNearbyVenues(names, latitudes, longitudes, radius=1500, LIMIT=120):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id'],
            v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue ID',
                  'Category ID' ]
    
    return(nearby_venues)



In [4]:
#generate the List of nearby Venues and its Category around Toronto
toronto_venues = getNearbyVenues(names=df3['Neighbourhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                               )

toronto_venues["Rating"] = np.nan
toronto_venues["Price"] = np.nan

toronto_venues.shape

(6833, 12)

In [5]:
#Filter out desired data based on the requirements from client, e.g. Restaurant, Bar, Asian Food  

toronto_venues = toronto_venues[toronto_venues['Venue Category'].str.contains("Restaurant|Diner|Bar|Beer|Pub|Brewery|Lounge")]
toronto_venues["Target_Restaurant"] = "No"

target_restaurant = toronto_venues[toronto_venues['Venue Category'].str.contains("Asian|Chinese|Cantone|Thai|Indian|Japanese|Filipino|Vietnamese|Korea|Hotpot|Shanghai|Hong Kong|Malay|Taiwanese|Turkish")]
bar = toronto_venues[toronto_venues['Venue Category'].str.contains("Bar|Beer|Pub|Brewery|Lounge")]


In [6]:
#Indicate the category of Venue, e.g. is it a Bar? Asian Restaurant? Other Restaurant?

for index in target_restaurant.index:
    toronto_venues.at[index,'Target_Restaurant'] = "Yes"

for index in bar.index:
    toronto_venues.at[index,'Target_Restaurant'] = "Bar"
    
toronto_venues = toronto_venues.reset_index(drop=True)


In [80]:
#\prepare statistical data for number of Restaurant (Asian and others) & Bar
data = toronto_venues.groupby(['Neighbourhood']).Target_Restaurant.value_counts().unstack(fill_value=0)
data['Sum'] = data['Yes'] + data['No'] + data['Bar']
data = data[(data['Sum'] > np.percentile(data['Sum'],85,axis=0))]
data['Bar%'] = (data['Bar']/data['Sum']) * 100
data['No%'] = (data['No']/data['Sum']) * 100
data['Yes%'] = (data['Yes']/data['Sum']) * 100
data['Yes_No_Ratio'] = data['Yes%'] / data['No%']
data['Bar_Restaurant_Ratio'] = data['Bar%'] / (data['Yes%'] + data['No%'])

#Categorized by k-Means Clustering
### set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(data)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]

# add clustering labels
data.insert(9, 'Cluster Labels', kmeans.labels_)

data = data.sort_values(by=['Cluster Labels'], ascending=True)

data

Target_Restaurant,Bar,No,Yes,Sum,Bar%,No%,Yes%,Yes_No_Ratio,Bar_Restaurant_Ratio,Cluster Labels
Neighbourhood,,,,,,,,,,
Business Reply Mail Processing Centre 969 Eastern,9,23,10,42,21.428571,54.761905,23.809524,0.434783,0.272727,0
"Cabbagetown, St. James Town",9,17,15,41,21.951220,41.463415,36.585366,0.882353,0.281250,0
Christie,11,18,10,39,28.205128,46.153846,25.641026,0.555556,0.392857,0
"Little Portugal, Trinity",17,21,10,48,35.416667,43.750000,20.833333,0.476190,0.548387,0
Studio District,12,19,9,40,30.000000,47.500000,22.500000,0.473684,0.428571,0
Church and Wellesley,7,25,6,38,18.421053,65.789474,15.789474,0.240000,0.225806,1
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",5,28,4,37,13.513514,75.675676,10.810811,0.142857,0.156250,1
Queen's Park,6,23,7,36,16.666667,63.888889,19.444444,0.304348,0.200000,1
Rosedale,5,26,6,37,13.513514,70.270270,16.216216,0.230769,0.156250,1


In [56]:
#Search data of "Rating" and "Price" to prioritize the places 
CLIENT_ID = 'E4PIB2UWOVN2RRZXH0RFHFW24TMK21H4NOCETLDZWMPFUFWZ' # Foursquare ID
CLIENT_SECRET = 'EC4UG1QFKT2PVELHCX1PI5H0NOYH0JCISTP5H3ORAE3XYDQU' 

for venue_id in toronto_venues_final['Venue ID']:
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    try: 
        toronto_venues_final.loc[toronto_venues_final["Venue ID"] == venue_id, 'Rating'] = requests.get(url).json()["response"]['venue']["rating"] 
    except KeyError:
        toronto_venues_final.loc[toronto_venues_final["Venue ID"] == venue_id, 'Rating'] = np.nan
    
    try: 
        toronto_venues_final.loc[toronto_venues_final["Venue ID"] == venue_id, 'Price'] = requests.get(url).json()["response"]['venue']["price"]['tier'] 
    except KeyError:
        toronto_venues_final.loc[toronto_venues_final["Venue ID"] == venue_id, 'Price'] = np.nan


In [ ]:
#Select Cluster = 0 as the target places, as it most-match client's requirement

data = data.loc[data['Cluster Labels'] == 0]
data = data.reset_index(drop=True)
loc_list=[]
for loc in data['Neighbourhood']:
    loc_list.append(loc)
    
#Prepare the list of Venues from the targeted places     
toronto_venues_final = toronto_venues.loc[toronto_venues['Neighbourhood'].isin(loc_list)]
toronto_venues_final = toronto_venues_final.reset_index(drop=True)
toronto_venues_final.shape

In [58]:
toronto_venues_final.head(4)
export_csv = toronto_venues_final.to_csv (r'C:\Users\Eden Cheng\Desktop\export_dataframe.csv', index = None, header=True)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Category ID,Rating,Price,Target_Restaurant
0,Studio District,43.659526,-79.340923,Tabule,43.659731,-79.346341,Middle Eastern Restaurant,519d414a498e6cc21f677ff4,4bf58dd8d48988d115941735,8.7,2.0,No
1,Studio District,43.659526,-79.340923,Completo,43.662550,-79.334049,Latin American Restaurant,53af0086498eb15122135e6a,4bf58dd8d48988d1be941735,8.7,2.0,No
2,Studio District,43.659526,-79.340923,Mean Bao,43.661417,-79.340209,Chinese Restaurant,55f07a51498ef8afddf53c9c,4bf58dd8d48988d145941735,7.7,1.0,Yes
3,Studio District,43.659526,-79.340923,The Comrade,43.659346,-79.347932,Bar,4aeb98fff964a520a6c321e3,4bf58dd8d48988d116941735,8.4,2.0,Bar
4,Studio District,43.659526,-79.340923,Ascari Enoteca,43.662054,-79.335457,Italian Restaurant,4ec03c1329c25ce5a22fca40,4bf58dd8d48988d110941735,8.3,2.0,No
5,Studio District,43.659526,-79.340923,I’ll Be Seeing You,43.659250,-79.348121,Bar,5696fec5498eec0b3ffec7be,4bf58dd8d48988d116941735,8.3,2.0,Bar
6,Studio District,43.659526,-79.340923,Ruby WatchCo.,43.659149,-79.349170,Restaurant,4bc37b7c920eb713b4851d2c,4bf58dd8d48988d1c4941735,8.4,2.0,No
7,Studio District,43.659526,-79.340923,White Lily Diner,43.658675,-79.351074,Diner,58233f576a80ef616dbf88ab,4bf58dd8d48988d147941735,8.8,1.0,No
8,Studio District,43.659526,-79.340923,Hi-Lo,43.659254,-79.348035,Dive Bar,5261df96498e25eda5a4432e,4bf58dd8d48988d118941735,8.0,2.0,Bar
9,Studio District,43.659526,-79.340923,Radical Road Brewing,43.662772,-79.332515,Brewery,57acc462cd10c32744f2d7cd,50327c8591d4c4b30a586d5d,8.3,2.0,Bar


In [85]:
#Combine the data of "Rating" and "Price" to the data set of target places, to judge which place shoudl we start our business

data1 = toronto_venues_final.groupby('Neighbourhood').mean().reset_index()
data1 = data1[['Neighbourhood','Rating','Price']]

data = data.loc[data['Cluster Labels'] == 0]
data = data.sort_values(by=['Neighbourhood'], ascending=True)
data1 = data1.sort_values(by=['Neighbourhood'], ascending=True)
data['Rating'] = data1['Rating'].values
data['Price'] = data1['Price'].values
data['Norm_Rating'] = (data['Rating'] - data['Rating'].min()) / (data['Rating'].max()-data['Rating'].min())
data['Norm_Price'] = (data['Price'] - data['Price'].min()) / (data['Price'].max()-data['Price'].min())
data['Scoring'] = data['Norm_Rating'] + data['Norm_Price']

data.sort_values(by=['Scoring'], ascending=False)

Target_Restaurant,Bar,No,Yes,Sum,Bar%,No%,Yes%,Yes_No_Ratio,Bar_Restaurant_Ratio,Cluster Labels,Rating,Price,Norm_Rating,Norm_Price,Scoring
Neighbourhood,,,,,,,,,,,,,,,
"Little Portugal, Trinity",17,21,10,48,35.416667,43.750000,20.833333,0.476190,0.548387,0,8.606250,2.170213,1.000000,1.000000,2.000000
Christie,11,18,10,39,28.205128,46.153846,25.641026,0.555556,0.392857,0,8.253846,2.027027,0.667861,0.685946,1.353807
Studio District,12,19,9,40,30.000000,47.500000,22.500000,0.473684,0.428571,0,8.055000,1.975000,0.480449,0.571833,1.052282
"Cabbagetown, St. James Town",9,17,15,41,21.951220,41.463415,36.585366,0.882353,0.281250,0,8.260976,1.714286,0.674580,0.000000,0.674580
Business Reply Mail Processing Centre 969 Eastern,9,23,10,42,21.428571,54.761905,23.809524,0.434783,0.272727,0,7.545238,2.000000,0.000000,0.626667,0.626667
